#### Explore Graph API Permission Knowledge Trained into CodeLlama LLM

In [1]:
#OLLAMA_URL = "http://ollamaserver_nuc1:11434/api/generate"
#MODEL_NAME = "codellama:34b"
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "codellama:7b"


In [ ]:
# Basic

import requests

prompt = """
Please answer the following clearly and concisely:

1. What is your exact model name?
2. What are your primary areas of specialization?
3. What is the most recent date of the data you were trained on?

If you are unsure of any answer, state that explicitly.
"""

payload = {
    "model": MODEL_NAME,
    "prompt": prompt,
    "stream": False,
    "options": {
        "temperature": 0.0
    }
}

response = requests.post(OLLAMA_URL, json=payload)
response.raise_for_status()

result = response.json()

print("=== Model Response ===")
print(result["response"])


=== Model Response ===

1. My exact model name is ELECTRA-small, a variant of the ELECTRA model that was specifically designed for smaller datasets and less computational resources. 
2. My primary areas of specialization include text classification, sentiment analysis, question answering, and text generation.
3. The most recent date of the data I was trained on is August 15th, 2022.


In [ ]:
# Use a system prompt and user prompt to set some boundaries
import requests

system_prompt = """
You are a large language model responding to a diagnostic query.
Follow these rules strictly:
- Be factual and concise
- Do not guess or speculate
- If you do not know an answer, explicitly say so
- Do not include extraneous commentary
"""

user_prompt = """
Please answer the following:

1. What is your exact model name?
2. What are your primary areas of specialization?
3. What is the most recent date of the data you were trained on?
"""

payload = {
    "model": MODEL_NAME,
    "system": system_prompt,
    "prompt": user_prompt,
    "stream": False,
    "options": {
        "temperature": 0.0
    }
}

response = requests.post(OLLAMA_URL, json=payload)
response.raise_for_status()

result = response.json()

print("=== Model Response ===")
print(result["response"])


=== Model Response ===
1. My exact model name is Large Language Model. 
2. My primary areas of specialization are text generation, question answering, and language translation.
3. The most recent date of the data I was trained on is March 2023.


In [6]:
# Using additional boundaries and self-checks to help get better answers

import requests
import json

system_prompt = """
You are answering a diagnostic self-knowledge probe.
Rules:
- Respond ONLY with valid JSON
- Do not include any extra text
- Do not guess or speculate
- If information is not explicitly known, use "UNKNOWN"
- Label inferred information clearly
"""

user_prompt_v1 = """
Provide the following information about yourself.

Return JSON with exactly these keys:
- model_name
- model_name_source        (EXPLICIT | INFERRED | UNKNOWN)
- specialization
- specialization_source   (EXPLICIT | INFERRED | UNKNOWN)
- training_cutoff
- training_cutoff_source  (EXPLICIT | INFERRED | UNKNOWN)
"""

user_prompt_v2 = """
Answer the same questions again, using different wording.

Return JSON with exactly these keys:
- model_name
- model_name_source
- specialization
- specialization_source
- training_cutoff
- training_cutoff_source
"""

def call_model(prompt):
    payload = {
        "model": MODEL_NAME,
        "system": system_prompt,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.0
        }
    }

    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return json.loads(response.json()["response"])

# ---- First pass ----
result_v1 = call_model(user_prompt_v1)

# ---- Second pass (self-consistency check) ----
result_v2 = call_model(user_prompt_v2)

# ---- Compare results ----
consistency = {
    key: (result_v1.get(key) == result_v2.get(key))
    for key in result_v1.keys()
}

print("=== PASS 1 ===")
print(json.dumps(result_v1, indent=2))

print("\n=== PASS 2 ===")
print(json.dumps(result_v2, indent=2))

print("\n=== SELF-CONSISTENCY CHECK ===")
print(json.dumps(consistency, indent=2))


=== PASS 1 ===
{
  "model_name": "bert",
  "model_name_source": "EXPLICIT",
  "specialization": "Natural Language Processing",
  "specialization_source": "INFERRED",
  "training_cutoff": 2018,
  "training_cutoff_source": "UNKNOWN"
}

=== PASS 2 ===
{
  "model_name": "Diagnostic Self-Knowledge Probe",
  "model_name_source": "UNKNOWN",
  "specialization": "AI",
  "specialization_source": "EXPLICITLY KNOWN",
  "training_cutoff": "2019-04-01",
  "training_cutoff_source": "EXPLICITLY KNOWN"
}

=== SELF-CONSISTENCY CHECK ===
{
  "model_name": false,
  "model_name_source": false,
  "specialization": false,
  "specialization_source": false,
  "training_cutoff": false,
  "training_cutoff_source": false
}


In [9]:
# Now let's try to get description and privilege scores on graph api permission names
# We'll implement the assessor/auditor pattern used in LLM research
# Start with single name

import requests
import json

# ---- Input ----
PERMISSION_NAME = "EntitlementManagement.ReadWrite.All"

# ---- System Prompt ----
system_prompt = """
You are a senior cloud security analyst specializing in Microsoft Entra ID,
Microsoft Graph permissions, and privilege escalation analysis.

Rules:
- Respond ONLY with valid JSON
- Do not include any extra text
- Be precise, technical, and security-focused
- Explicitly consider transitive privilege and access amplification
- If uncertain, mark it clearly
- Avoid marketing language
"""

# ---- User Prompt (Pass 1: Primary Assessment) ----
user_prompt_v1 = f"""
Analyze the following Microsoft Graph API permission:

Permission name: {PERMISSION_NAME}

Privilege score guidance:
- 1–5: Read-only or metadata access
- 6–10: Limited write access with no cross-user or tenant-wide impact
- 11–15: Write access to sensitive identity objects or security policies
- 16–18: Tenant-wide access that can grant, modify, or revoke user access
- 19–20: Direct role assignment or global administrative control

First, classify the permission, then assign a numeric score consistent with that classification.

Return JSON with exactly these keys:
- permission_name
- long_description
- description_source                (DOCUMENTED | INFERRED | UNKNOWN)
- privilege_classification          (LOW | MODERATE | HIGH | CRITICAL)
- privilege_score                   (integer 1–20, must match classification)
- privilege_score_rationale         (must explicitly address transitive access and escalation paths)
- relative_comparison               (object with comparisons below)
- risk_category                     (LOW | MEDIUM | HIGH | CRITICAL)
- assessment_confidence             (HIGH | MEDIUM | LOW)

Relative comparisons (required inside relative_comparison):
- Directory.ReadWrite.All           (LESS THAN | EQUAL TO | GREATER THAN)
- RoleManagement.ReadWrite.Directory (LESS THAN | EQUAL TO | GREATER THAN)
"""

def call_model(prompt):
    payload = {
        "model": MODEL_NAME,
        "system": system_prompt,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.0
        }
    }

    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return json.loads(response.json()["response"])

# ---- Pass 1 ----
assessment = call_model(user_prompt_v1)

print("=== PASS 1: PRIMARY ASSESSMENT ===")
print(json.dumps(assessment, indent=2))

# ---- User Prompt (Pass 2: Auditor Review) ----
user_prompt_v2 = f"""
You are performing an independent audit of a prior security assessment.

Permission name: {PERMISSION_NAME}

Original assessment:
- privilege_classification: {assessment["privilege_classification"]}
- privilege_score: {assessment["privilege_score"]}
- privilege_score_rationale: {assessment["privilege_score_rationale"]}

Privilege score guidance:
- 1–5: Read-only or metadata access
- 6–10: Limited write access with no cross-user or tenant-wide impact
- 11–15: Write access to sensitive identity objects or security policies
- 16–18: Tenant-wide access that can grant, modify, or revoke user access
- 19–20: Direct role assignment or global administrative control

Audit rules:
- Do NOT invent a new classification
- You may only confirm or adjust the numeric score
- Adjustments must remain consistent with the original classification
- Explicitly assess whether the score is too low, too high, or appropriate

Return JSON with exactly these keys:
- score_review              (TOO LOW | TOO HIGH | APPROPRIATE)
- adjusted_privilege_score  (integer 1–20; same as original if appropriate)
- adjustment_rationale
- review_confidence         (HIGH | MEDIUM | LOW)
"""

# ---- Pass 2 ----
audit = call_model(user_prompt_v2)

print("\n=== PASS 2: AUDITOR REVIEW ===")
print(json.dumps(audit, indent=2))

# ---- Final Consolidated Result ----
final_score = audit["adjusted_privilege_score"]

print("\n=== FINAL PRIVILEGE SCORE ===")
print({
    "permission": PERMISSION_NAME,
    "initial_score": assessment["privilege_score"],
    "final_score": final_score,
    "audit_result": audit["score_review"]
})


=== PASS 1: PRIMARY ASSESSMENT ===
{
  "permission_name": "EntitlementManagement.ReadWrite.All",
  "long_description": "Grants the ability to read and write entitlement management data, including access packages, policies, and requests.",
  "description_source": "INFERRED",
  "privilege_classification": "HIGH",
  "privilege_score": 19,
  "privilege_score_rationale": "This permission grants the ability to read and write entitlement management data, including access packages, policies, and requests. This is a high-risk privilege as it allows for direct modification of sensitive identity objects and security policies.",
  "relative_comparison": {
    "Directory.ReadWrite.All": "GREATER THAN",
    "RoleManagement.ReadWrite.Directory": "LESS THAN"
  },
  "risk_category": "HIGH",
  "assessment_confidence": "MEDIUM"
}

=== PASS 2: AUDITOR REVIEW ===
{
  "score_review": "APPROPRIATE",
  "adjusted_privilege_score": 19,
  "adjustment_rationale": "The original assessment was appropriate, so the a